In [2]:
import numpy as np
import pandas as pd
import xlrd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,\
ExtraTreesClassifier, VotingClassifier, StackingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder,\
OrdinalEncoder, MaxAbsScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score,\
accuracy_score, classification_report, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import xgboost as xgb
from matplotlib import pyplot as plt
import warnings

In [3]:
season_df = pd.read_csv('/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/WRegularSeasonDetailedResults.csv')
season_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70002,2023,113,3268,96,3234,68,H,0,34,71,...,38,11,15,18,19,14,24,5,3,14
70003,2023,113,3385,69,3163,64,A,0,23,63,...,21,12,15,11,25,14,12,5,7,19
70004,2023,113,3416,57,3396,53,H,0,19,60,...,26,14,17,16,19,5,9,10,9,20
70005,2023,113,3437,67,3177,64,H,0,24,67,...,19,8,11,21,26,13,12,9,2,14


In [4]:
tourney_df = pd.read_csv('/Users/aheinke/Documents/Flatiron/NYC-DS-010923/Phase_3/Phase3_Proj/PROJ_CSVs/WNCAATourneyDetailedResults.csv')
tourney_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,...,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,...,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,...,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,...,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,...,26,11,17,16,22,9,10,3,4,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2022,147,3163,91,3301,87,N,2,37,77,...,23,16,19,6,30,20,13,4,7,16
756,2022,147,3257,62,3276,50,N,0,25,58,...,14,15,20,10,24,9,21,6,2,12
757,2022,151,3163,63,3390,58,N,0,21,57,...,23,8,13,11,23,10,11,11,3,16
758,2022,151,3376,72,3257,59,N,0,27,57,...,8,4,7,11,18,5,15,13,2,17


In [5]:
merged_df = pd.concat([season_df, tourney_df])

# sort the merged dataframe by "col_name" in ascending order
sorted_df = merged_df.sort_values(['Season', 'DayNum'], ascending=[True, True])

# reset the index
games_df = sorted_df.reset_index(drop=True)
games_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70762,2023,113,3268,96,3234,68,H,0,34,71,...,38,11,15,18,19,14,24,5,3,14
70763,2023,113,3385,69,3163,64,A,0,23,63,...,21,12,15,11,25,14,12,5,7,19
70764,2023,113,3416,57,3396,53,H,0,19,60,...,26,14,17,16,19,5,9,10,9,20
70765,2023,113,3437,67,3177,64,H,0,24,67,...,19,8,11,21,26,13,12,9,2,14


In [6]:
stanford_df = games_df[(games_df['WTeamID'] == 3390) | (games_df['LTeamID'] == 3390)]
stanford_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
81,2010,11,3390,89,3330,56,A,0,32,65,...,16,10,18,19,19,8,21,9,1,20
187,2010,13,3390,81,3353,66,A,0,27,71,...,11,11,16,10,23,14,13,3,7,19
344,2010,17,3390,99,3337,50,H,0,40,63,...,21,7,11,17,16,7,17,12,0,15
518,2010,20,3390,76,3413,51,A,0,28,57,...,15,4,7,8,12,11,15,7,1,14
721,2010,25,3390,60,3428,41,H,0,21,62,...,11,3,5,4,23,9,17,1,2,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70051,2023,97,3449,72,3390,67,H,0,25,58,...,16,12,13,2,20,16,15,3,3,18
70191,2023,101,3390,84,3112,60,A,0,35,59,...,17,7,10,8,15,14,12,7,2,12
70386,2023,104,3390,96,3113,64,A,0,35,76,...,16,15,21,10,18,11,11,7,4,20
70562,2023,109,3390,50,3425,47,H,0,21,59,...,23,11,12,17,20,7,6,5,4,10


In [7]:
stanford_df.value_counts('LTeamID')

LTeamID
3390    69
3143    27
3112    25
3425    25
3417    25
        ..
3257     1
3253     1
3245     1
3243     1
3110     1
Length: 93, dtype: int64

In [ ]:
# Split your data into features and target variable
X = stanford_df.drop('target', axis=1)
y = stanford_df['target']

# Define your XGBoost model and parameters
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8
)

# Define your cross-validation strategy
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate your model using cross-validation
scores = cross_val_score(model, X, y, cv=kfold)

# Print the average and standard deviation of the scores
print("Mean score: {:.2f}".format(scores.mean()))
print("Standard deviation: {:.2f}".format(scores.std()))

In [ ]:
X = stanford_df.drop('target', axis=1)
y = stanford_df['target']

model = xgb.XGBRegressor()

# define the cross-validation procedure
cv = KFold(n_splits=5, shuffle=True, random_state=123)

# fit the model using cross-validation
scores = xgb.cv(model.get_params(), xgb.DMatrix(X), nfold=5, metrics='rmse', seed=123)

# calculate the mean cross-validation score
mean_score = scores['test-rmse-mean'].mean()
print('Mean RMSE:', mean_score)